In [1]:
import math
import os
from collections import defaultdict, deque
from typing import Deque

import torch
import torch.nn as nn
from torchmetrics.image.fid import FrechetInceptionDistance
from torchvision import transforms

In [2]:
# This UNET-style prediction model was originally included as part of the Score-based generative modelling tutorial
# by Yang Song et al: https://colab.research.google.com/drive/120kYYBOVa1i0TD85RjlEkFjaWDxSFUx3?usp=sharing

import numpy as np


class GaussianFourierProjection(nn.Module):
    """Gaussian random features for encoding time steps."""

    def __init__(self, embed_dim, scale=30.0):
        super().__init__()
        # Randomly sample weights during initialization. These weights are fixed
        # during optimization and are not trainable.
        self.W = nn.Parameter(torch.randn(embed_dim // 2) * scale, requires_grad=False)

    def forward(self, x):
        x_proj = x[:, None] * self.W[None, :] * 2 * np.pi
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)


class Dense(nn.Module):
    """A fully connected layer that reshapes outputs to feature maps."""

    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.dense = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.dense(x)[..., None, None]


class ScoreNet(nn.Module):
    """A time-dependent score-based model built upon U-Net architecture."""

    def __init__(self, marginal_prob_std, channels=[32, 64, 128, 256], embed_dim=256):
        """Initialize a time-dependent score-based network.

        Args:
          marginal_prob_std: A function that takes time t and gives the standard
            deviation of the perturbation kernel p_{0t}(x(t) | x(0)).
          channels: The number of channels for feature maps of each resolution.
          embed_dim: The dimensionality of Gaussian random feature embeddings.
        """
        super().__init__()
        # Gaussian random feature embedding layer for time
        self.embed = nn.Sequential(
            GaussianFourierProjection(embed_dim=embed_dim),
            nn.Linear(embed_dim, embed_dim),
        )
        # Encoding layers where the resolution decreases
        self.conv1 = nn.Conv2d(1, channels[0], 3, stride=1, bias=False)
        self.dense1 = Dense(embed_dim, channels[0])
        self.gnorm1 = nn.GroupNorm(4, num_channels=channels[0])
        self.conv2 = nn.Conv2d(channels[0], channels[1], 3, stride=2, bias=False)
        self.dense2 = Dense(embed_dim, channels[1])
        self.gnorm2 = nn.GroupNorm(32, num_channels=channels[1])
        self.conv3 = nn.Conv2d(channels[1], channels[2], 3, stride=2, bias=False)
        self.dense3 = Dense(embed_dim, channels[2])
        self.gnorm3 = nn.GroupNorm(32, num_channels=channels[2])
        self.conv4 = nn.Conv2d(channels[2], channels[3], 3, stride=2, bias=False)
        self.dense4 = Dense(embed_dim, channels[3])
        self.gnorm4 = nn.GroupNorm(32, num_channels=channels[3])

        # Decoding layers where the resolution increases
        self.tconv4 = nn.ConvTranspose2d(
            channels[3], channels[2], 3, stride=2, bias=False
        )
        self.dense5 = Dense(embed_dim, channels[2])
        self.tgnorm4 = nn.GroupNorm(32, num_channels=channels[2])
        self.tconv3 = nn.ConvTranspose2d(
            channels[2] + channels[2],
            channels[1],
            3,
            stride=2,
            bias=False,
            output_padding=1,
        )
        self.dense6 = Dense(embed_dim, channels[1])
        self.tgnorm3 = nn.GroupNorm(32, num_channels=channels[1])
        self.tconv2 = nn.ConvTranspose2d(
            channels[1] + channels[1],
            channels[0],
            3,
            stride=2,
            bias=False,
            output_padding=1,
        )
        self.dense7 = Dense(embed_dim, channels[0])
        self.tgnorm2 = nn.GroupNorm(32, num_channels=channels[0])
        self.tconv1 = nn.ConvTranspose2d(channels[0] + channels[0], 1, 3, stride=1)

        # The swish activation function
        self.act = lambda x: x * torch.sigmoid(x)
        self.marginal_prob_std = marginal_prob_std

    def forward(self, x, t):
        # Obtain the Gaussian random feature embedding for t
        embed = self.act(self.embed(t))
        # Encoding path
        h1 = self.conv1(x)
        ## Incorporate information from t
        h1 += self.dense1(embed)
        ## Group normalization
        h1 = self.gnorm1(h1)
        h1 = self.act(h1)
        h2 = self.conv2(h1)
        h2 += self.dense2(embed)
        h2 = self.gnorm2(h2)
        h2 = self.act(h2)
        h3 = self.conv3(h2)
        h3 += self.dense3(embed)
        h3 = self.gnorm3(h3)
        h3 = self.act(h3)
        h4 = self.conv4(h3)
        h4 += self.dense4(embed)
        h4 = self.gnorm4(h4)
        h4 = self.act(h4)

        # Decoding path
        h = self.tconv4(h4)
        ## Skip connection from the encoding path
        h += self.dense5(embed)
        h = self.tgnorm4(h)
        h = self.act(h)
        h = self.tconv3(torch.cat([h, h3], dim=1))
        h += self.dense6(embed)
        h = self.tgnorm3(h)
        h = self.act(h)
        h = self.tconv2(torch.cat([h, h2], dim=1))
        h += self.dense7(embed)
        h = self.tgnorm2(h)
        h = self.act(h)
        h = self.tconv1(torch.cat([h, h1], dim=1))

        # Normalize output
        h = h / self.marginal_prob_std(t)[:, None, None, None]
        return h

In [3]:
# ExponentialMovingAverage implementation as used in pytorch vision
# https://github.com/pytorch/vision/blob/main/references/classification/utils.py#L159

# BSD 3-Clause License

# Copyright (c) Soumith Chintala 2016,
# All rights reserved.

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# * Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.

# * Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.

# * Neither the name of the copyright holder nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


class ExponentialMovingAverage(torch.optim.swa_utils.AveragedModel):
    """Maintains moving averages of model parameters using an exponential decay.
    ``ema_avg = decay * avg_model_param + (1 - decay) * model_param``
    `torch.optim.swa_utils.AveragedModel <https://pytorch.org/docs/stable/optim.html#custom-averaging-strategies>`_
    is used to compute the EMA.
    """

    def __init__(self, model, decay, device="cpu"):
        def ema_avg(avg_model_param, model_param, num_averaged):
            return decay * avg_model_param + (1 - decay) * model_param

        super().__init__(model, device, ema_avg, use_buffers=True)

In [4]:
import matplotlib.pyplot as plt
from torchvision import datasets, utils
from tqdm.auto import tqdm


class DDPM(nn.Module):
    def __init__(
        self,
        network,
        predict_using="epsilon",
        scheduler_method="linear",
        variance_reduction="none",
        T=100,
        beta_1=1e-4,
        beta_T=2e-2,
    ):
        """
        Initialize Denoising Diffusion Probabilistic Model

        Parameters
        ----------
        network: nn.Module
            The inner neural network used by the diffusion process. Typically a Unet.
        beta_1: float
            beta_t value at t=1
        beta_T: [float]
            beta_t value at t=T (last step)
        T: int
            The number of diffusion steps.
        """

        super(DDPM, self).__init__()

        # Normalize time input before evaluating neural network
        # Reshape input into image format and normalize time value before sending it to network model
        self._network = network
        self.network = lambda x, t: (
            self._network(x.reshape(-1, 1, 28, 28), (t.squeeze() / T))
        ).reshape(-1, 28 * 28)

        # Total number of time steps
        self.T = T

        # Initialize the history with a default deque of maxlen 10 for each new timestep
        self.loss_history: defaultdict[int, Deque[float]] = defaultdict(
            lambda: deque(maxlen=10)
        )

        # Variance reduction: none, low_discrepancy
        self.variance_reduction = variance_reduction

        # Predict using epsilon, mu or x_0
        self.predict_using = predict_using

        # Scheduler method: linear or cosine
        self.scheduler_method = scheduler_method

        if self.scheduler_method == "linear":
            self.register_buffer("beta", torch.linspace(beta_1, beta_T, T + 1))
        elif self.scheduler_method == "cosine":
            # scheduler = schedulers.DDPMScheduler(
            #     num_train_timesteps=self.T, beta_schedule="squaredcos_cap_v2"
            # )
            # self.register_buffer("beta", scheduler.betas)
            self.register_buffer("beta", self.cosine_variance_schedule(T + 1))

        # Registering as buffers to ensure they get transferred to the GPU automatically

        self.register_buffer("alpha", 1 - self.beta)
        self.register_buffer("alpha_bar", self.alpha.cumprod(dim=0))

    @staticmethod
    def cosine_variance_schedule(timesteps, s=0.008):
        steps = torch.linspace(0, timesteps, steps=timesteps + 1)
        f_t = torch.cos(((steps / timesteps + s) / (1.0 + s)) * math.pi / 2.0) ** 2
        return torch.clip(1.0 - f_t[1:] / f_t[:timesteps], 0.0, 0.999)

    def forward_diffusion(self, x0, t, epsilon):
        """
        q(x_t | x_0)
        Forward diffusion from an input datapoint x0 to an xt at timestep t, provided a N(0,1) noise sample epsilon.
        Note that we can do this operation in a single step

        Parameters
        ----------
        x0: torch.tensor
            x value at t=0 (an input image)
        t: int
            step index
        epsilon:
            noise sample

        Returns
        -------
        torch.tensor
            image at timestep t
        """

        mean = torch.sqrt(self.alpha_bar[t]) * x0
        std = torch.sqrt(1 - self.alpha_bar[t])

        return mean + std * epsilon

    def reverse_diffusion(self, xt, t, epsilon):
        """
        p(x_{t-1} | x_t)
        Single step in the reverse direction, from x_t (at timestep t) to x_{t-1}, provided a N(0,1) noise sample epsilon.

        Parameters
        ----------
        xt: torch.tensor
            x value at step t
        t: int
            step index
        epsilon:
            noise sample

        Returns
        -------
        torch.tensor
            image at timestep t-1
        """

        # Eq 11 in Ho et al, 2020
        if self.predict_using == "epsilon":
            mean = (
                1.0
                / torch.sqrt(self.alpha[t])
                * (
                    xt
                    - (self.beta[t])
                    / torch.sqrt(1 - self.alpha_bar[t])
                    * self.network(xt, t)
                )
            )

            std = torch.where(
                t > 0,
                torch.sqrt(
                    ((1 - self.alpha_bar[t - 1]) / (1 - self.alpha_bar[t]))
                    * self.beta[t]
                ),
                0,
            )

        elif self.predict_using == "x0":
            nn_predicted_x0 = self.network(xt, t)  ## NN prediction of x0

            # Index alpha, alpha_bar, alpha_bar_pred,  beta
            alpha_t = self.alpha[t]
            alpha_bar_t = self.alpha_bar[t]
            alpha_bar_prev_t_1 = self.alpha_bar[t - 1]
            beta_t = self.beta[t]

            # Equation 7 in DDPM by Ho et al, 2020
            coefficient_x0 = torch.sqrt(alpha_bar_prev_t_1) / (1 - alpha_bar_t) * beta_t
            coefficient_xt = (
                torch.sqrt(alpha_t) * (1 - alpha_bar_prev_t_1) / (1 - alpha_bar_t)
            )
            mean = coefficient_x0 * nn_predicted_x0 + coefficient_xt * xt

            beta_tilde = ((1 - alpha_bar_prev_t_1) / (1 - alpha_bar_t)) * beta_t
            std = torch.sqrt(beta_tilde)

        return mean + std * epsilon

    @torch.no_grad()
    def sample(self, shape):
        """
        Sample from diffusion model (Algorithm 2 in Ho et al, 2020)

        Parameters
        ----------
        shape: tuple
            Specify shape of sampled output. For MNIST: (nsamples, 28*28)

        Returns
        -------
        torch.tensor
            sampled image
        """

        # Sample xT: Gaussian noise
        xT = torch.randn(shape).to(self.beta.device)

        xt = xT
        for t in range(self.T, 0, -1):
            noise = torch.randn_like(xT) if t > 1 else 0
            t = torch.tensor(t).expand(xt.shape[0], 1).to(self.beta.device)
            xt = self.reverse_diffusion(xt, t, noise)

        return xt

    def weight_function(self, t: torch.Tensor) -> torch.Tensor:
        """
        Importance sampling weight function based on E[L_t^2] using a history.
        """
        t = t.squeeze().long()
        weights = []
        for timestep in t:
            timestep_item = timestep.item()
            history = self.loss_history[timestep_item]
            if history:
                # Compute E[L_t^2] as the mean of the last 10 values
                mean_history = sum(history) / len(history)
                weight = math.sqrt(mean_history)
            else:
                # Default weight if no history is available
                weight = 1.0
            weights.append(weight)

        # Normalize weights
        # norm = np.linalg.norm(weights, dtype=np.float32)

        # weights = weights / weights.sum()  # Normalize weights

        return torch.tensor(weights, device=t.device)

    def update_loss_squared_history(self, t: torch.Tensor, loss: torch.Tensor) -> None:
        """
        Update the history of L_t^2 for each timestep.
        """
        t = t.squeeze().long()
        # Calculate L_t^2 for the given timesteps
        loss_squared = (loss.mean(dim=1)) ** 2  # Mean over batch dimension
        for timestep, lsq in zip(t, loss_squared):
            timestep_item = timestep.item()
            lsq_value = lsq.item()
            self.loss_history[timestep_item].append(lsq_value)

    def elbo_simple(self, x0):
        """
        ELBO training objective (Algorithm 1 in Ho et al, 2020)

        Parameters
        ----------
        x0: torch.tensor
            Input image

        Returns
        -------
        float
            ELBO value
        """

        # Sample time step t
        if self.variance_reduction == "low_discrepancy":
            t = self.generate_low_discrepancy_timesteps(x0.shape[0], self.T, x0.device)
        elif self.variance_reduction == "none":
            t = torch.randint(1, self.T, (x0.shape[0], 1)).to(x0.device)
        elif self.variance_reduction == "importance_sampling":
            t = torch.randint(1, self.T, (x0.shape[0], 1)).to(x0.device)

        # Sample noise
        epsilon = torch.randn_like(x0)

        # TODO: Forward diffusion to produce image at step t
        xt = self.forward_diffusion(x0, t, epsilon)

        if self.predict_using == "epsilon":
            target = epsilon
        elif self.predict_using == "x0":
            target = x0

        return -nn.MSELoss(reduction="none")(target, self.network(xt, t))

        # if self.variance_reduction == "importance_sampling":
        #     weights = self.weight_function(t.float())
        #     weights = weights / weights.sum()  # Normalize weights
        #     self.update_loss_squared_history(t, loss)

        #     # torch.Size([256, 784]) torch.Size([256])
        #     # shape of loss is (batch_size, 784) and shape of weights is (batch_size)
        #     # wonder if this is the correct way to apply the weights??
        #     foo = (loss.mean(dim=1) / weights).mean()  # Apply reweighting
        #     bar = (loss.mean(dim=1) * weights.squeeze()).mean()

        #     loss = (loss.mean(dim=1) * weights.squeeze()).mean()

        # return -bar
        # return -nn.MSELoss(reduction="mean")(target, self.network(xt, t))

        # t = torch.randint(1, self.T, (x0.shape[0], 1)).to(x0.device)
        # # Importance weight for each timestep
        # weights = weight_function(self, t.float())
        # weights = weights / weights.sum()  # Normalize weights

        # # Sample noise
        # epsilon = torch.randn_like(x0)
        # # Forward diffusion to produce image at step t
        # xt = self.forward_diffusion(x0, t, epsilon)
        # # Compute loss for predicting noise
        # loss = nn.MSELoss(reduction='none')(epsilon, self.network(xt, t))
        # # Update history of L_t^2
        # update_loss_squared_history(self, t, loss)
        # # Apply importance weights
        # loss = (loss.mean(dim=1) * weights.squeeze()).mean()
        # return -loss

        # correction by importance sampling
        # p_t = self.get_p_t()[t].to(x0.device)  # Get p_t for sampled t
        # corrected_loss = (loss / p_t).mean()  # Apply reweighting

    def generate_low_discrepancy_timesteps(self, batch_size, T, device):
        """
        Generates timesteps for low-discrepancy in the ELBO calculation.
        This is used to reduce the variance and the approach is the one
        described in the paper Variational Diffusion Models by Kingma et el.

        This method is described in appendix I.1 of the paper.

        Parameters
        ----------
        batch_size : int
            The batch size (number of samples).
        T : int
            Total number of diffusion steps.
        device : torch.device
            Device for storing the tensor.

        Returns
        -------
        torch.tensor
            Low-discrepancy timesteps `t` of shape (batch_size, 1).
        """
        # Sample a single uniform random number u0 ~ U(0, 1)
        u0 = torch.rand(1, device=device).item()
        # Generate timesteps using low-discrepancy sampling
        ts = (u0 + torch.arange(batch_size, device=device) / batch_size) % 1
        # Ensure timesteps are integers in the range [1, T]
        ts = (ts * T).long().unsqueeze(1)

        return ts

    def loss(self, x0):
        """
        Loss function. Just the negative of the ELBO.
        """

        return -self.elbo_simple(x0).mean()


## end of DDPM class


def train(
    model,
    optimizer,
    scheduler,
    dataloader,
    epochs,
    device,
    ema=True,
    per_epoch_callback=None,
):
    """
    Training loop

    Parameters
    ----------
    model: nn.Module
        Pytorch model
    optimizer: optim.Optimizer
        Pytorch optimizer to be used for training
    scheduler: optim.LRScheduler
        Pytorch learning rate scheduler
    dataloader: utils.DataLoader
        Pytorch dataloader
    epochs: int
        Number of epochs to train
    device: torch.device
        Pytorch device specification
    ema: Boolean
        Whether to activate Exponential Model Averaging
    per_epoch_callback: function
        Called at the end of every epoch
    """

    # Setup progress bar
    total_steps = len(dataloader) * epochs
    progress_bar = tqdm(range(total_steps), desc="Training")

    if ema:
        ema_global_step_counter = 0
        ema_steps = 10
        ema_adjust = dataloader.batch_size * ema_steps / epochs
        ema_decay = 1.0 - 0.995
        ema_alpha = min(1.0, (1.0 - ema_decay) * ema_adjust)
        ema_model = ExponentialMovingAverage(
            model, device=device, decay=1.0 - ema_alpha
        )

    for epoch in range(epochs):
        # Switch to train mode
        model.train()

        global_step_counter = 0
        for i, (x, _) in enumerate(dataloader):
            x = x.to(device)
            optimizer.zero_grad()
            loss = model.loss(x)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Update progress bar
            progress_bar.set_postfix(
                loss=f"⠀{loss.item():12.4f}",
                epoch=f"{epoch + 1}/{epochs}",
                lr=f"{scheduler.get_last_lr()[0]:.2E}",
            )
            progress_bar.update()

            if ema:
                ema_global_step_counter += 1
                if ema_global_step_counter % ema_steps == 0:
                    ema_model.update_parameters(model)

        if per_epoch_callback:
            per_epoch_callback(ema_model.module if ema else model)


# Parameters
T = 1000
learning_rate = 1e-3
epochs = 100
batch_size = 256


# Rather than treating MNIST images as discrete objects, as done in Ho et al 2020,
# we here treat them as continuous input data, by dequantizing the pixel values (adding noise to the input data)
# Also note that we map the 0..255 pixel values to [-1, 1], and that we process the 28x28 pixel values as a flattened 784 tensor.
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Lambda(
            lambda x: x + torch.rand(x.shape) / 255
        ),  # Dequantize pixel values
        transforms.Lambda(lambda x: (x - 0.5) * 2.0),  # Map from [0,1] -> [-1, -1]
        transforms.Lambda(lambda x: x.flatten()),
    ]
)

# Download and transform train dataset
dataloader_train = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data", download=True, train=True, transform=transform),
    batch_size=batch_size,
    shuffle=True,
)


def reporter(model):
    """Callback function used for plotting images during training"""

    # Switch to eval mode
    model.eval()

    with torch.no_grad():
        nsamples = 10
        samples = model.sample((nsamples, 28 * 28)).cpu()

        # Map pixel values back from [-1,1] to [0,1]
        samples = (samples + 1) / 2
        samples = samples.clamp(0.0, 1.0)

        # Plot in grid
        grid = utils.make_grid(samples.reshape(-1, 1, 28, 28), nrow=nsamples)
        plt.gca().set_axis_off()
        plt.imshow(transforms.functional.to_pil_image(grid), cmap="gray")
        plt.show()


/home/carstenj/dev/pml/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Select device
# Check for CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
# Check for MPS (for Apple Silicon)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


# predict_by = ["epsilon", "x0"]
# schedules = ["cosine", "linear"]
# variance_reductions = ["none", "low_discrepancy"]

predict_by = ["epsilon"]
schedules = ["linear"]
variance_reductions = ["importance_sampling"]

# loop over all combinations of predict_by and schedules
for predict_using in predict_by:
    for scheduler_method in schedules:
        for variance_reduction in variance_reductions:
            print(
                f"Training model with predict_using={predict_using}, scheduler_method={scheduler_method}, and variance_reduction={variance_reduction}"
            )

            # Skip loop if model already exists
            if (
                f"./models/ddpm_{predict_using}_{scheduler_method}_{variance_reduction}_{device}.pt"
                in os.listdir("./models")
            ):
                print("Model already exists. Skipping training.")
                continue

            # Construct Unet
            # The original ScoreNet expects a function with std for all the
            # different noise levels, such that the output can be rescaled.
            # Since we are predicting the noise (rather than the score), we
            # ignore this rescaling and just set std=1 for all t.
            mnist_unet = ScoreNet((lambda t: torch.ones(1).to(device)))

            # Construct model
            model = DDPM(
                mnist_unet,
                T=T,
                predict_using=predict_using,
                scheduler_method=scheduler_method,
                variance_reduction=variance_reduction,
            ).to(device)

            # Construct optimizer
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            # Setup simple scheduler
            scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9999)

            # Call training loop
            train(
                model,
                optimizer,
                scheduler,
                dataloader_train,
                epochs=epochs,
                device=device,
                ema=True,
                per_epoch_callback=reporter,
            )

            # Save model
            torch.save(
                model.state_dict(),
                f"./models/ddpm_{predict_using}_{scheduler_method}_{variance_reduction}_{device}.pt",
            )


In [5]:
def calculate_fid(model, dataloader, device, num_samples=None):
    """Calculate FID score between real and generated images"""
    fid = FrechetInceptionDistance(normalize=True).to(torch.float32).to(device)

    if not num_samples:
        num_samples = 10000

    transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),  # Resize to 299x299 pixels
        ]
    )

    # Generate fake images
    model.eval()
    model.to(device)

    with torch.no_grad():
        for real_images, _ in dataloader:
            batch_size = real_images.shape[0]
            samples = model.sample((batch_size, 28 * 28))
            samples = samples.view(-1, 1, 28, 28)  # Reshape to (N, 1, 28, 28)
            samples = (samples + 1) / 2  # Map from [-1, 1] to [0, 1]
            samples = samples.repeat(1, 3, 1, 1)
            samples = transform(samples)  # Resize to (N, 3, 299, 299)
            fid.update(samples.to(device), real=False)
            if fid.real_features_num_samples >= num_samples:
                break
            # Get real images
            real_images = real_images.view(-1, 1, 28, 28)
            real_images = (real_images + 1) / 2
            real_images = real_images.repeat(
                1, 3, 1, 1
            )  # Repeat channels to get (N, 3, 28, 28)
            real_images = transform(real_images)  # Resize to (N, 3, 299, 299)
            fid.update(real_images.to(device), real=True)

    fid_score = fid.compute()

    return float(fid_score)

In [12]:
def calculate_fid_v2(model, dataloader, device, num_samples=10000):
    """
    Calculate the Fréchet Inception Distance (FID) score between real and generated images.

    Parameters:
        model: The generative model used to produce images.
        dataloader: DataLoader providing batches of real images.
        device: The device (CPU or GPU) to perform computations on.
        num_samples: The number of samples to use for FID calculation (default is 10,000).

    Returns:
        fid_score: The calculated FID score as a floating-point number.
    """

    # Initialize FID metric
    fid_metric = FrechetInceptionDistance(normalize=True).to(device)

    # Transformation to resize images to 299x299 (required for Inception model)
    transform = transforms.Resize((299, 299))

    # Set the model to evaluation mode
    model.eval()
    model.to(device)

    # Keep track of the number of processed samples
    total_fake_samples = 0
    total_real_samples = 0

    with torch.no_grad():
        # Generate and collect fake images
        while total_fake_samples < num_samples:
            batch_size = min(dataloader.batch_size, num_samples - total_fake_samples)

            # Generate fake images using the model
            fake_images = model.sample((batch_size, 28 * 28))
            # Reshape fake images to (N, 1, 28, 28) to allow for 3 channels
            fake_images = fake_images.view(-1, 1, 28, 28)
            # Map from [-1, 1] to [0, 1]
            fake_images = (fake_images + 1) / 2
            # Convert to 3 channels
            fake_images = fake_images.repeat(1, 3, 1, 1)
            # Resize to (N, 3, 299, 299)
            fake_images = transform(fake_images)

            # Update FID metric with fake images
            fid_metric.update(fake_images.to(device), real=False)
            total_fake_samples += batch_size

        # Collect real images
        for real_images, _ in dataloader:
            batch_size = real_images.size(0)

            # Only process up to num_samples
            if total_real_samples + batch_size > num_samples:
                batch_size = num_samples - total_real_samples
                real_images = real_images[:batch_size]

            # Process real images
            real_images = real_images.view(-1, 1, 28, 28)
            real_images = (real_images + 1) / 2  # Map from [-1, 1] to [0, 1]
            real_images = real_images.repeat(1, 3, 1, 1)  # Convert to 3 channels
            real_images = transform(real_images)  # Resize to (N, 3, 299, 299)

            # Update FID metric with real images
            fid_metric.update(real_images.to(device), real=True)
            total_real_samples += batch_size

            if total_real_samples >= num_samples:
                break

    # Compute FID score
    fid_score = fid_metric.compute()
    return float(fid_score)


In [14]:
# predict_by = ["epsilon", "x0"]
# schedules = ["cosine", "linear"]
# variance_reductions = ["none", "low_discrepancy", "importance_sampling"]
# device = torch.device("cuda")

predict_by = ["epsilon"]
schedules = ["linear"]
variance_reductions = ["none"]

# loop over all combinations of predict_by and schedules
for predict_using in predict_by:
    for scheduler_method in schedules:
        for variance_reduction in variance_reductions:
            print(
                f"Generating samples from model with predict_using={predict_using}, scheduler_method={scheduler_method}, and variance_reduction={variance_reduction}"
            )

            # Construct Unet
            # The original ScoreNet expects a function with std for all the
            # different noise levels, such that the output can be rescaled.
            # Since we are predicting the noise (rather than the score), we
            # ignore this rescaling and just set std=1 for all t.
            mnist_unet = ScoreNet((lambda t: torch.ones(1).to(device)))

            # Construct model
            model = DDPM(
                mnist_unet,
                T=T,
                predict_using=predict_using,
                scheduler_method=scheduler_method,
                variance_reduction=variance_reduction,
            )

            # Load model from checkpoint using predict_using, scheduler_method, and variance_reduction
            checkpoint_path = f"./models/ddpm_{predict_using}_{scheduler_method}_{variance_reduction}_cuda.pt"

            model.load_state_dict(torch.load(checkpoint_path, weights_only=True))

            # reporter(model)

            fid = calculate_fid_v2(model, dataloader_train, device, num_samples=1280)
            print(fid)


Generating samples from model with predict_using=epsilon, scheduler_method=linear, and variance_reduction=none
321.75567626953125
